In [7]:
import sys
from pathlib import Path

project_root = Path("..").resolve()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("Project root added to sys.path:", project_root)


Project root added to sys.path: /Users/tharungopinath/Desktop/Q-A Agentic AI


In [8]:
from dotenv import load_dotenv
import os
load_dotenv()
print("GROQ_API_KEY loaded:", bool(os.getenv("GROQ_API_KEY")))


GROQ_API_KEY loaded: True


In [10]:
import requests, os
BASE = os.getenv("QA_AGENT_URL", "http://localhost:8000")
print("Base URL:", BASE)

def send(message, user_id="notebook"):
    r = requests.post(f"{BASE}/chat", json={"message": message, "user_id": user_id})
    r.raise_for_status()
    return r.json()

print("Example: factual query ->")
print(send("What is Python?"))

print("\nExample: conversational query ->")
print(send("How are you today?"))

Base URL: http://localhost:8000
Example: factual query ->
{'answer': '(tool) Python is a high-level, interpreted programming language known for readability.', 'source': 'tool', 'memory': [{'role': 'user', 'text': 'What is Python?'}, {'role': 'assistant', 'text': '(tool) Python is a high-level, interpreted programming language known for readability.'}]}

Example: conversational query ->
{'answer': "I'm functioning properly. How can I assist you today?", 'source': 'llm', 'memory': [{'role': 'user', 'text': 'What is Python?'}, {'role': 'assistant', 'text': '(tool) Python is a high-level, interpreted programming language known for readability.'}, {'role': 'user', 'text': 'How are you today?'}, {'role': 'assistant', 'text': "I'm functioning properly. How can I assist you today?"}]}


In [11]:
def clear_mem(user_id="notebook"):
    r = requests.post(f"{BASE}/clear_memory", params={"user_id": user_id})
    r.raise_for_status()
    return r.json()

print(clear_mem("notebook"))

{'status': 'cleared', 'user_id': 'notebook'}


In [12]:
from agents.fact_detection import is_factual, small_search_tool
from agents.interface import answer_question
from typing import Dict, Any

def chatbot_node(state: Dict[str, Any]) -> Dict[str, Any]:
    raw = state.get("messages", "")

    if isinstance(raw, list):
        query = raw[-1] if raw else ""
        if not isinstance(query, str):
            query = str(query)
    else:
        query = str(raw)

    try:
        if is_factual(query):
            tool_ans = small_search_tool(query)
            if tool_ans:
                return {"messages": f"(tool) {tool_ans}"}
    except Exception:
        pass  

    try:
        reply = answer_question(query, memory=[])
        reply_text = reply if isinstance(reply, str) else str(reply)
    except Exception as e:
        reply_text = f"LLM error: {e}"

    return {"messages": reply_text}


In [ ]:
from typing import Dict, Any
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

from agents import fact_detection as fact_mod  
from agents import interface as interface_mod   
from agents import memory as memory_mod        

is_factual = getattr(fact_mod, "is_factual", None)
small_search_tool = getattr(fact_mod, "small_search_tool", None)
answer_question = getattr(interface_mod, "answer_question", None)
Memory = getattr(memory_mod, "Memory", None)


class State(TypedDict):
    messages: list  
    intent: str     


def router_decision(state: Dict[str, Any]) -> str:
    raw = state.get("messages", "")
    if isinstance(raw, list) and raw:
        q = str(raw[-1]).lower()
    else:
        q = str(raw).lower()

    try:
        if is_factual and is_factual(q):
            return "technical_node"
    except Exception:
        pass
    return "llm_node"


def interface_node(state: Dict[str, Any]) -> Dict[str, Any]:

    raw = state.get("messages", "")
    query = raw[-1] if isinstance(raw, list) and raw else raw or ""
    query = str(query)

    
    reply = ""
    try:
        if answer_question:
            
            reply = answer_question(query, memory=[])
            
            if isinstance(reply, str) and len(reply) > 300:
                reply = reply.split("\n")[0] + "..."
    except Exception:
        reply = ""

    intent = router_decision({"messages": query})
    return {"messages": reply, "intent": intent}


def technical_node(state: Dict[str, Any]) -> Dict[str, Any]:

    raw = state.get("messages", "")
    
    query = raw if isinstance(raw, str) else (raw[-1] if isinstance(raw, list) and raw else "")
    query = str(query)

   
    try:
        if small_search_tool:
            tool_ans = small_search_tool(query)
            if tool_ans:
                return {"messages": f"(tool) {tool_ans}"}
    except Exception:
        pass

    
    try:
        if answer_question:
            return {"messages": answer_question(query, memory=[])}
    except Exception as e:
        return {"messages": f"LLM error: {e}"}

    return {"messages": "technical: no handler available."}


def llm_node(state: Dict[str, Any]) -> Dict[str, Any]:
    raw = state.get("messages", "")
    query = raw if isinstance(raw, str) else (raw[-1] if isinstance(raw, list) and raw else "")
    query = str(query)

    try:
        if answer_question:
            return {"messages": answer_question(query, memory=[])}
    except Exception as e:
        return {"messages": f"LLM error: {e}"}

    return {"messages": "llm: no handler available."}



graph_builder = StateGraph(State)


graph_builder.add_node("interface_node", interface_node)
graph_builder.add_node("fact_detection_node", technical_node)
graph_builder.add_node("llm_node", llm_node)


graph_builder.add_edge(START, "interface_node")


graph_builder.add_conditional_edges(
    "interface_node",
    lambda st: st.get("intent", "llm_node"),
    {
        "technical_node": "technical_node",
        "llm_node": "llm_node"
    }
)

graph_builder.add_edge("technical_node", END)
graph_builder.add_edge("llm_node", END)


try:
    graph = graph_builder.compile()
except Exception:
    graph = graph_builder  

print("Compiled graph type:", type(graph))

try:
    png = graph.get_graph().draw_mermaid_png()
    from pathlib import Path
    out = Path("..") / "static"
    out.mkdir(parents=True, exist_ok=True)
    (out / "agentic_graph_two_node.png").write_bytes(png)
    print("Saved mermaid PNG to static/agentic_graph_two_node.png")
except Exception as e:
    print("Could not auto-render mermaid PNG (langgraph version may differ):", e)


print("\n== quick invokes ==")
try:
    print("Billing-like test -> (should route to technical if factual):",
          graph.invoke({"messages": "What is FastAPI?"}).get("messages"))
except Exception as e:
    print("invoke failed:", e)

try:
    print("Conversational test -> (should route to llm):",
          graph.invoke({"messages": "Tell me a joke."}).get("messages"))
except Exception as e:
    print("invoke failed:", e)


Compiled graph type: <class 'langgraph.graph.state.CompiledStateGraph'>
Saved mermaid PNG to static/agentic_graph_two_node.png

== quick invokes ==
Billing-like test -> (should route to technical if factual): That's correct. FastAPI is known for its high-performance capabilities, scalability, and ease of use. It supports Python 3.7+ and is built on standard Python type hints, making it a popular choice for building APIs quickly and efficiently.
Conversational test -> (should route to llm): That's a classic joke. The humor comes from the double meaning of "make up" - both referring to atoms being the basic building blocks of matter and "make up" as in to fabricate or lie. Scientists do trust atoms, they're a fundamental part of chemistry and physics.
